In [16]:
import requests
from lxml import html
from bs4 import BeautifulSoup
import re
import json

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import gspread
from oauth2client.service_account import ServiceAccountCredentials

In [17]:
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('project-100-reviews-6063049ddd77.json', scope)
client = gspread.authorize(creds)
def GetSpreadsheetData(sheetNameurl, worksheetIndex): 
    sheet = client.open_by_url(sheetNameurl).get_worksheet(worksheetIndex) 
    return sheet.get_all_values()[1:]

In [18]:
spreadsheet = GetSpreadsheetData('https://docs.google.com/spreadsheets/d/13Tg3hMg5gvRycZ9eD93NnYcMCUm5BVgnB3SvvP2Hcco/edit?'+
                                 'usp=sharing', 0)

In [19]:
spreadsheet_df = pd.DataFrame(spreadsheet, columns=['form_timestamp', 'full_name', 'email', 'group_name', 'business_name1', 
                                                    'visit_date1', 'reviewurl', 'cocoapreneursubmit', 'pledge'])
google_df = spreadsheet_df[spreadsheet_df['reviewurl'].str.contains('https://goo\.gl/maps/.*')]

In [20]:
def get_metacontent(url):
    resp = requests.get(url)
    soup = BeautifulSoup(resp.content, 'lxml')
    metas = soup.find_all('meta')
    content_array = [i.get('content') for i in metas]
    return content_array

In [21]:
#star rating and review description
def get_review(content_arr):
    star_rating0, review_desc0 = [], []
    for c in content_arr:
        match = re.match(r'[★]+', c)
        if match != None:
            star_rating0.append(len(match[0]))
            review_desc0.append(re.findall(r'[^\'★☆].*', c))
    star_rating0 = np.mean(star_rating0)
    review_desc0 = review_desc0[0][0][1:].replace('"', "")
    return star_rating0, review_desc0

In [22]:
#reviewer name and business name
def get_review_names(content_arr):
    reviewer_name0, business_name0 = [], []
    for c in content_arr: 
        match = re.findall(r'Google review of (.*?) by (.*)', c)
        if match != []:
            reviewer_name0.append(match[0][0])
            business_name0.append(match[0][1])
    return reviewer_name0[0], business_name0[0]

In [23]:
def GoogleReviewScraper(urls):
    star_rating, review_desc, reviewer_name, business_name = [], [], [], []
    for url in urls:
        content = get_metacontent(url)
        review, names = get_review(content), get_review_names(content)
        star_rating.append(review[0])
        review_desc.append(review[1])
        business_name.append(names[0])
        reviewer_name.append(names[1])
    review_data = {'business_name2': business_name, 'reviewer_name': reviewer_name,'star_rating': star_rating, \
                   'review_desc': review_desc, 'review_platform': 'Google', 'reviewurl':urls}
    return pd.DataFrame(review_data)

In [24]:
reviews_df0 = GoogleReviewScraper(google_df['reviewurl'])
reviews_df0.head()

,business_name2,reviewer_name,star_rating,review_desc,review_platform,reviewurl
4,Sichuan Gourmet,Thilo Krah-Tomala,5.0,Great Chinese eatery with spicy Sichuan food. ...,Google,https://goo.gl/maps/81AHC82LizDEuf2w9
5,Sichuan Gourmet,Shauna Bigi,1.0,Placed an order for take out...amazing crispy ...,Google,https://goo.gl/maps/FHBkpX2xwGb5HFSK8
6,Sichuan Gourmet,Jen C,4.0,Some dishes were great but others were miss . ...,Google,https://goo.gl/maps/cqBhkSw8Q7tN34Fv8
7,Sichuan Gourmet,Jason Chen,2.0,i think they have changed the chief. The taste...,Google,https://goo.gl/maps/65NqTgESVgc8S2hZ9
8,Leon's Caribbean Food,Andi Halim,5.0,Had the goat curry and oxtail plates along wit...,Google,https://goo.gl/maps/htQVrTNeBUDZusvy7


In [25]:
reviews_df = google_df.merge(reviews_df0, on="reviewurl")[['form_timestamp','full_name', 'reviewer_name','email','group_name',
                                                           'business_name1', 'business_name2','visit_date1', 
                                                           'review_platform','reviewurl','review_desc', 'star_rating', 
                                                           'cocoapreneursubmit','pledge']]
reviews_df

,form_timestamp,full_name,reviewer_name,email,group_name,business_name1,business_name2,visit_date1,review_platform,reviewurl,review_desc,star_rating,cocoapreneursubmit,pledge
0,8/9/2020 12:06:21,Dummy Data,Thilo Krah-Tomala,dummydata@gmail.com,CAASI Grief to Action,Sichuan Gourmet,Sichuan Gourmet,7/19/2020,Google,https://goo.gl/maps/81AHC82LizDEuf2w9,Great Chinese eatery with spicy Sichuan food. ...,5.0,Cannot find the business on the Cocoapreneur,I do not plan on continuing to support Project...
1,8/10/2020 12:06:21,Dummy Data,Shauna Bigi,dummydata@gmail.com,CAASI Grief to Action,Sichuan Gourmet,Sichuan Gourmet,7/20/2020,Google,https://goo.gl/maps/FHBkpX2xwGb5HFSK8,Placed an order for take out...amazing crispy ...,1.0,Cannot find the business on the Cocoapreneur,I do not plan on continuing to support Project...
2,8/11/2020 12:06:21,Dummy Data,Jen C,dummydata@gmail.com,CAASI Grief to Action,Sichuan Gourmet,Sichuan Gourmet,7/21/2020,Google,https://goo.gl/maps/cqBhkSw8Q7tN34Fv8,Some dishes were great but others were miss . ...,4.0,Cannot find the business on the Cocoapreneur,I do not plan on continuing to support Project...
3,8/12/2020 12:06:21,Dummy Data,Jason Chen,dummydata@gmail.com,CAASI Grief to Action,Sichuan Gourmet,Sichuan Gourmet,7/22/2020,Google,https://goo.gl/maps/65NqTgESVgc8S2hZ9,i think they have changed the chief. The taste...,2.0,Cannot find the business on the Cocoapreneur,In the next two weeks
4,8/5/2020 12:06:21,Dummy Data,Andi Halim,dummydata@gmail.com,CAASI Grief to Action,Leon's Caribbean Restaurant,Leon's Caribbean Food,7/15/2020,Google,https://goo.gl/maps/htQVrTNeBUDZusvy7,Had the goat curry and oxtail plates along wit...,5.0,Cannot find the business on the Cocoapreneur,In the next month


In [26]:
reviews_df.to_csv('googlereviews_df.csv', index=False)